# 获取全量的股票编码


In [11]:
#参数及工具函数部分
import pandas as pd
import numpy as np
from jqfactor import *
from jqfactor import get_factor_values

from jqdata import *

import warnings  
warnings.filterwarnings('ignore')


#获取数据主函数
#输入股票池、指标名称、开始日期、结束日期
#返回行标签为日期，列表签为股票名称的dataframe表格
def get_factor_data(stockPool, factor,date_start, date_end):
    
    #获取股票池函数
    def get_stock(stockPool, begin_date):
        if stockPool == 'HS300':#用于获取沪深300股票池
            stockList = get_index_stocks('000300.XSHG', begin_date)
        elif stockPool == 'ZZ500':#用于获取中证500股票池
            stockList = get_index_stocks('399905.XSHE', begin_date)
        elif stockPool == 'ZZ800':#用于获取中证800股票池
            stockList = get_index_stocks('399906.XSHE', begin_date)   
        elif stockPool == 'A':#用于获取全部A股股票池
            stockList = get_index_stocks('000002.XSHG', begin_date) + get_index_stocks('399107.XSHE', begin_date)
        else:#自定义输入股票池
            stockList = stockPool
        return stockList    
    
    #从财务库获取数据
    def get_factor_data1(factor,stock, date):
        if factor in val:
            q = query(valuation).filter(valuation.code.in_(stock))
            df = get_fundamentals(q, date)
        elif factor in bal:
            q = query(balance).filter(balance.code.in_(stock))
            df = get_fundamentals(q, date)
        elif factor in cf:
            q = query(cash_flow).filter(cash_flow.code.in_(stock))
            df = get_fundamentals(q, date)
        elif factor in inc:
            q = query(income).filter(income.code.in_(stock))
            df = get_fundamentals(q, date)
        elif factor in ind:
            q = query(indicator).filter(indicator.code.in_(stock))
            df = get_fundamentals(q, date)

        df.index = df['code']
        data = pd.DataFrame(index = df.index)
        data[date] = df[factor]

        return data.T
    
    #获取日期列表
    date_list = get_trade_days(start_date = date_start, end_date = date_end)
    #空df预备存储数据
    data = pd.DataFrame(columns = get_stock(stockPool,begin_date=date_list[-1]))
    
    #获取五张财务基础所有指标名称
    val = get_fundamentals(query(valuation).limit(1)).columns.tolist()
    bal = get_fundamentals(query(balance).limit(1)).columns.tolist()
    cf = get_fundamentals(query(cash_flow).limit(1)).columns.tolist()
    inc = get_fundamentals(query(income).limit(1)).columns.tolist()
    ind = get_fundamentals(query(indicator).limit(1)).columns.tolist()
    all_columns = val+bal+cf+inc+ind
    
    
    # 判断相应的股票池
    # 如果stockpool是字母，则按照字母提取
    # 如果stockpool是数组，则直接赋值字母
    # 确定相应的股票池
    if isinstance(stockPool,list):
        all_stocks = stockPool
    else:
        all_stocks = get_stock(stockPool, date_end)
        
    #循环时间列表获取指标数据
    for date in date_list:
        #获取因子数据
        if factor in all_columns: #可以从财务库直接取到因子值的因子
            data_temp = get_factor_data1(factor,all_stocks, date)
        else: #可以从因子库直接取到因子值的因子
            try:
                data_temp = get_factor_values(all_stocks, [factor], end_date = date, count = 1)[factor]
            except:
                print('系统暂不能获取该因子，请获取其他因子')
                break
        data = pd.concat([data, data_temp], axis = 0) 
    return data



In [12]:
# 确定
begin_date = "2019-11-1"
end_date = "2019-11-1"

# 000002：A股指数
# 399107：深证A指

stockList = get_index_stocks('000002.XSHG', begin_date) + get_index_stocks('399107.XSHE', begin_date)

In [13]:
len(set(stockList))

3651

In [14]:
# 最终生成的pd数组
# 存储所有数据

factor_pd = pd.DataFrame(index = stockList)
factor_pd.head()

""
600000.XSHG
600004.XSHG
600006.XSHG
600007.XSHG
600008.XSHG


# 添加相应的价格数据


In [15]:
factor_pd['price'] = np.array(get_price(stockList, count = 1  , end_date = end_date , fields=['close'])['close'].T)

In [16]:
factor_pd.head()

,price
600000.XSHG,12.75
600004.XSHG,18.11
600006.XSHG,4.44
600007.XSHG,17.67
600008.XSHG,3.34


# 添加股票名称、行业数据

In [17]:
factor_pd['name'] = get_all_securities()['display_name']
factor_pd.head()

,price,name
600000.XSHG,12.75,浦发银行
600004.XSHG,18.11,白云机场
600006.XSHG,4.44,东风汽车
600007.XSHG,17.67,中国国贸
600008.XSHG,3.34,首创股份


In [18]:
# 提取行业原始数据
industry_pd_temp = pd.DataFrame.from_dict(get_industry(stockList, date=end_date), orient='index')['zjw']

# 遍历series，转化为包含股票编码、行业信息的数组
industry_list = [[d[0],d[1]['industry_name']] for d in industry_pd_temp.items()]

# 重新生成新的pandas数据，准备进行合并
industry_pd = pd.DataFrame(data = industry_list,columns = ['code','industry'])
industry_pd = industry_pd.set_index('code')

# 进行赋值
factor_pd['industry'] = industry_pd['industry']

factor_pd.head()

,price,name,industry
600000.XSHG,12.75,浦发银行,货币金融服务
600004.XSHG,18.11,白云机场,航空运输业
600006.XSHG,4.44,东风汽车,汽车制造业
600007.XSHG,17.67,中国国贸,房地产业
600008.XSHG,3.34,首创股份,水的生产和供应业


# 获取财务报表因子值


In [19]:

# 设置因子池
factor_pool = [
    'pe_ratio',                # 动态市盈率 
    'pe_ratio_lyr',            # 静态市盈率
    'pb_ratio',                # 市净率
    
    'market_cap',              # 市值
    'circulating_market_cap',  # 流通市值
    
    'roe_ttm',                 # 12个月滚动的ROE
    'ROAEBITTTM',              # （利润总额（TTM）+利息支出（TTM）） / 总资产在过去12个月的平均

    'debt_to_asset_ratio',     # 债务总资产比=负债合计/总资产     
    'debt_to_equity_ratio',    # 产权比率=负债合计/归属母公司所有者权益合计
    'MLEV',                    # 市场杠杆=非流动负债合计/(非流动负债合计+总市值)
    'long_debt_to_working_capital_ratio',        # 长期负债与营运资金比率=非流动负债合计/(流动资产合计-流动负债合计)
    'fixed_asset_ratio',       # 固定资产总额:固定资产比率=(固定资产+工程物资+在建工程)/总资产
    'intangible_asset_ratio',  # 无形资产比率=(无形资产+研发支出+商誉)/总资产
    
    
    'net_profit_ratio',        # 净利润（TTM）/营业收入（TTM）
    'net_profit_to_total_operate_revenue_ttm',   # 净利润与营业总收入之比TTM
    'net_non_operating_income_to_total_profit',  # 营业外收支利润净额/利润总额
    'gross_income_ratio',      # 销售毛利率=(营业收入（TTM）-营业成本（TTM）)/营业收入（TTM）
    'adjusted_profit_to_total_profit',           # 扣除非经常损益后的净利润/利润总额
    'total_profit_to_cost_ratio',                # 成本费用利润率=利润总额/(营业成本+财务费用+销售费用+管理费用)，以上科目使用的都是TTM的数值
    
    
    
    'total_asset_turnover_rate',                 # 总资产周转率=营业收入(ttm)/总资产
    'accounts_payable_turnover_days',            # 应付账款周转天数，360/应付账款周转率
    'account_receivable_turnover_days',          # 应收账款周转天数，360/应收账款周转率
    'inventory_turnover_days', # 存货周转天数
    'OperatingCycle',          # 应收账款周转天数+存货周转天数
    'inventory_turnover_rate', # 存货周转率=营业成本（TTM）/存货  
    
    
    
    'DEGM',                    # 毛利率增长率
    'operating_profit_growth_rate',              # 营业利润增长率=(今年营业利润（TTM）/去年营业利润（TTM）)-1
    'sale_expense_to_operating_revenue',         # 销售费用（TTM）/营业总收入（TTM）
    'operating_tax_to_operating_revenue_ratio_ttm',      # 营业税金及附加（TTM）/营业收入（TTM）
    
    
    'net_operate_cash_flow_to_operate_income',   # 经营活动产生的现金流量净额（TTM）/(营业总收入（TTM）-营业总成本（TTM）
    'net_operating_cash_flow_coverage',          # 经营活动产生的现金流量净额/归属于母公司所有者的净利润
    'cash_rate_of_sales',                        # 经营活动产生的现金流量净额（TTM） / 营业收入（TTM）
    'net_operate_cash_flow_to_total_liability',  # 经营活动产生的现金流量净额/负债合计
    'ACCA',                                      # 现金流资产比-资产回报率,其中现金流资产比=经营活动产生的现金流量净额/总资产
    
    'current_ratio',           # 流动比率=流动资产合计/流动负债合计
    'quick_ratio',             # 速动比率=(流动资产合计-存货)/ 流动负债合计
    'super_quick_ratio',       # （货币资金+交易性金融资产+应收票据+应收帐款+其他应收款）／流动负债合计
    
    
    'LVGI',                    # 本期(年报)资产负债率/上期(年报)资产负债率
    'SGI',                     # 本期(年报)营业收入/上期(年报)营业收入
    'GMI',                     # 上期(年报)毛利率/本期(年报)毛利率
    'DSRI',                    # 本期(年报)应收账款占营业收入比例/上期(年报)应收账款占营业收入比例
    'SGAI',                    # 本期(年报)销售管理费用占营业收入的比例/上期(年报)销售管理费用占营业收入的比例
    
    
    'DEGM_8y',                 # 过去8年(1+DEGM)的累成 ^ (1/8) - 1
    'maximum_margin',          # max(margin_stability, DEGM_8y)
    'margin_stability'         # mean(GM)/std(GM); GM 为过去8年毛利率ttm
    ]

In [20]:
for factor_i in factor_pool:

    print("正在处理：%s"%str(factor_i))

    # 从函数提取的原始值
    # 注意，因为选择的是iloc[0]，所以实际选择的是begin_date
    factor_raw = get_factor_data(stockList, factor_i, begin_date, end_date)
    
    # 1、生成临时pd数组进行存储
    factor_temp = pd.DataFrame()
    
    # 2、设置factor_temp的code
    factor_temp['code'] = factor_raw.T.index
    
    # 3、设置factor_temp对应的值，列名时对应的因子名
    factor_temp[factor_i] = np.array(factor_raw.T.iloc[:,0])
    
    factor_temp = factor_temp.set_index('code')

    # 4、合并相应的值
    factor_pd = pd.concat([factor_pd,factor_temp],axis = 1)
        
    

正在处理：pe_ratio
正在处理：pe_ratio_lyr
正在处理：pb_ratio
正在处理：market_cap
正在处理：circulating_market_cap
正在处理：roe_ttm
正在处理：ROAEBITTTM
正在处理：debt_to_asset_ratio
正在处理：debt_to_equity_ratio
正在处理：MLEV
正在处理：long_debt_to_working_capital_ratio
正在处理：fixed_asset_ratio
正在处理：intangible_asset_ratio
正在处理：net_profit_ratio
正在处理：net_profit_to_total_operate_revenue_ttm
正在处理：net_non_operating_income_to_total_profit
正在处理：gross_income_ratio
正在处理：adjusted_profit_to_total_profit
正在处理：total_profit_to_cost_ratio
正在处理：total_asset_turnover_rate
正在处理：accounts_payable_turnover_days
正在处理：account_receivable_turnover_days
正在处理：inventory_turnover_days
正在处理：OperatingCycle
正在处理：inventory_turnover_rate
正在处理：DEGM
正在处理：operating_profit_growth_rate
正在处理：sale_expense_to_operating_revenue
正在处理：operating_tax_to_operating_revenue_ratio_ttm
正在处理：net_operate_cash_flow_to_operate_income
正在处理：net_operating_cash_flow_coverage
正在处理：cash_rate_of_sales
正在处理：net_operate_cash_flow_to_total_liability
正在处理：ACCA
正在处理：current_ratio
正在处理：quick_ratio
正在处理：sup

In [21]:
factor_pd[factor_pd.index == '000895.XSHE']

,price,name,industry,pe_ratio,pe_ratio_lyr,pb_ratio,market_cap,circulating_market_cap,roe_ttm,ROAEBITTTM,debt_to_asset_ratio,debt_to_equity_ratio,MLEV,long_debt_to_working_capital_ratio,fixed_asset_ratio,intangible_asset_ratio,net_profit_ratio,net_profit_to_total_operate_revenue_ttm,net_non_operating_income_to_total_profit,gross_income_ratio,adjusted_profit_to_total_profit,total_profit_to_cost_ratio,total_asset_turnover_rate,accounts_payable_turnover_days,account_receivable_turnover_days,inventory_turnover_days,OperatingCycle,inventory_turnover_rate,DEGM,operating_profit_growth_rate,sale_expense_to_operating_revenue,operating_tax_to_operating_revenue_ratio_ttm,net_operate_cash_flow_to_operate_income,net_operating_cash_flow_coverage,cash_rate_of_sales,net_operate_cash_flow_to_total_liability,ACCA,current_ratio,quick_ratio,super_quick_ratio,LVGI,SGI,GMI,DSRI,SGAI,DEGM_8y,maximum_margin,margin_stability
000895.XSHE,30.37,双汇发展,农副食品加工业,19.3788,20.5121,6.6365,1008.066,268.0319,0.341522,0.06775,0.398282,0.695842,0.002622,0.065004,0.408683,0.0313,0.099628,0.099532,0.000587,0.199436,0.76426,0.138416,2.043764,16.024542,8.66341,64.342652,73.006062,5.595044,-0.016049,0.069371,0.050842,0.005562,0.685712,0.742339,0.078335,0.108424,-0.020117,1.395568,0.642457,0.567897,1.131707,0.966699,0.883065,0.763483,1.102166,NaN,NaN,5.808419


In [22]:
factor_pd.to_csv('2019q3.csv')

In [23]:
q = query(
        valuation.code,
        valuation.market_cap,
        valuation.pe_ratio,
        indicator.roe
    ).filter(
    valuation.code == '000001.XSHE',
    )

rets = [get_fundamentals(q, statDate='2014q'+str(i)) for i in range(1, 5)]

In [24]:
rets[0]

,code,market_cap,pe_ratio,roe
0,000001.XSHE,1025.3843,6.7322,4.41


In [25]:
rets[1]

,code,market_cap,pe_ratio,roe
0,000001.XSHE,1132.2072,6.7813,4.21


In [26]:
rets[2]

,code,market_cap,pe_ratio,roe
0,000001.XSHE,1158.4843,6.5186,4.54


In [27]:
rets[3]

,code,market_cap,pe_ratio,roe
0,000001.XSHE,1809.7032,9.4113,3.19
